## Learning PySpark in 10 minutes  

The dataframe in PySpark is used extensively for cleaning of files dumped in AWS S3 buckets. The cleaned dataframe is then written back to the bucket in CSV or parquet format.

I have used the famous Oracle `scott/tiger` datasets of ``EMP`` and ```DEPT```. I believe that working with 14 rows of known data gives an easier understanding of what is changing in the underlying dataset.

The 2 files of `emp.csv` and 'dept.csv' are also present in here so easy access. I have changed the formats of the hiredate column to make cleaning noticeable.



In [1]:
print('hello world')

hello world
